In [13]:
import numpy as np 
import pandas as pd 
import scipy.ndimage
import os
from scipy.signal import butter, filtfilt
def butter_highpass(cutoff, fs, order=5):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = butter(order, normal_cutoff, btype='high', analog=False)
    return b, a

def butter_highpass_filter(data, cutoff, fs, order=5):
    b, a = butter_highpass(cutoff, fs, order=order)
    y = filtfilt(b, a, data)
    return y

In [16]:
directory =  r"C:\Users\sergi\OneDrive - Imperial College London\Files\Code Files\Dopamine CNN detection dataset\5HT_DATASET\Comparison Dataset/Colorplots"
all_files = os.listdir(directory)   # imagine you're one directory above test dir
txt_files = list(filter(lambda x: x[-4:] == '.txt', all_files))
CVs = []
for file in txt_files: 
    matrix = open(directory+"/"+file).read()
    matrix = np.array([item.split() for item in matrix.split('\n')[:-1]]).astype('float32')
    #Normalize the size of the matrix to 200x600.
    matrix = scipy.ndimage.zoom(matrix, [200/matrix.shape[0], 1])
    for i in range(0, matrix.shape[0]): 
        matrix[i] = butter_highpass_filter(matrix[i], 0.05, 10, order=5)
        
    matrixes_5s = np.hsplit(matrix, matrix.shape[1]/1)
    counter = 0
    for c in matrixes_5s:
        np.savetxt(file.split(".")[0]+'-n'+str(counter)+'.txt', c, delimiter="\t")
        counter = counter + 1